The fitness function can be found in the UDP file (*fitness_function*). In the following, we will have a closer look at it, explain its arguments, and show-case how to create a custom fitness function.

The fitness function looks as follows:

In [ ]:
ALPHA = 0.1
BETA = 1 - ALPHA
def fitness_function(cube_ensemble, steps_fraction, offset, num_cube_types, init_cube_types, target_cube_types, target_cubes):
    '''
    Fitness function used to evaluate a chromosome.

    Measures how well the final cube configuration fits the target configuration.
    Takes also into account how fast the final cube configuration has been reached (i.e., the number of pivoting operations)!
    Note: cubes of the same type are interchangable without affecting the fitness.

    Args:
        cube_ensemble: the ProgrammableCubes Object containing the cube positions.
        steps_fraction: number of pivots required divided by number of maximum pivots allowed.
        num_cube_types: number of different cube types.
        init_cube_types: list containing cube type of each initial cube. Sorted the same way as initial cube positions.
        cube_types: list containing cube type of each target cube. Sorted the same way as target cube positions.
        target_cubes: list of target cube positions. 
    Returns:
        score: fitness of the chromosome (float)
    '''
    num_correct_cubes = 0
    num_total_cubes = len(cube_ensemble.cube_position)
    for types in range(num_cube_types):
        target_list = target_cubes[target_cube_types==types].tolist()
        final_list = cube_ensemble.cube_position[init_cube_types==types].tolist()
        overlap = [cube in final_list for cube in target_list]
        num_correct_cubes += np.sum(overlap)
    cube_fraction = num_correct_cubes / num_total_cubes
    score = BETA * cube_fraction + ALPHA * (1-steps_fraction)
    score = (score-offset)/(1-offset)
    
    return score

*init_cube_types*, *target_cube_types*, and *target_cubes* are all provided in the data folders. For instance, for the ISS:

In [3]:
import numpy as np
init_cube_types = np.load('./data/ISS/Initial_Cube_Types.npy')
target_cube_types = np.load('./data/ISS/Target_Cube_Types.npy')
target_cubes = np.load('./data/ISS/Target_Config.npy')

*offset* and *num_cube_types* are obtained from the json file of the problem:

In [4]:
import json
with open('./problems/ISS.json', 'r') as infile:
    problem_file = json.load(infile)
offset = problem_file['fitness_offset']
num_cube_types = problem_file['num_cube_types']

*cube_ensemble* is your ProgrammableCubes object instance (after applying the chromosome), and *steps_fraction* is obtained from the effective length of your chromosome:

In [12]:
chromosome = np.array([0,1,20,3,-1, 0, 0, 0, -1])

steps_needed = int(np.where(chromosome == -1)[0][0]/2)
max_length = problem_file['max_cmds']

steps_fraction = steps_needed/max_length

#### Creating custom fitness functions

You might want to tweak the fitness function during optimization. The following loss function is an example of a customized fitness that can be used in isolation with a ProgrammableCubes object instance. It only returns the fraction of cubes that are in the right place. 

You can also replace the fitness function in the UDP file; however, in this case, you will have to use the same argument structure as the original function for it to work! Do not forget to change it back to obtain the score you would receive on optimize!

In [8]:
def custom_fitness_function(cube_ensemble, problem = 'ISS'):
    # Load some stuff
    with open('./problems/{}.json'.format(problem), 'r') as infile:
        num_cube_types = json.load(infile)['num_cube_types']
    init_cube_types = np.load('./data/{}/Initial_Cube_Types.npy'.format(problem))
    target_cube_types = np.load('./data/{}/Target_Cube_Types.npy'.format(problem))
    target_cubes = np.load('./data/{}/Target_Config.npy'.format(problem))
    
    # Calculate cube overlap with target configuration
    num_correct_cubes = 0
    num_total_cubes = len(cube_ensemble.cube_position)
    for types in range(num_cube_types):
        target_list = target_cubes[target_cube_types==types].tolist()
        final_list = cube_ensemble.cube_position[init_cube_types==types].tolist()
        overlap = [cube in final_list for cube in target_list]
        num_correct_cubes += np.sum(overlap)
    cube_fraction = num_correct_cubes / num_total_cubes
    
    # calculate the score
    score = cube_fraction
    
    return score

For simplicity, you could also split the part calculating the cube overlap and the actual score in two separate functions.

Lets test our custom fitness function :)

In [11]:
from programmable_cubes_UDP import ProgrammableCubes

# simple function for creating a random chromosome
def create_random_chromosome():
    chrom = []
    for i in range(6000):
        # select a cube randomly
        cube_id = np.random.randint(148)
        # select a maneuver randomly
        move = np.random.randint(6)
        
        # add command to chromosome
        chrom += [cube_id, move]
    # chromosome always ends with -1
    chrom += [-1]
    return np.array(chrom)

# create a random chromosome
chromosome = create_random_chromosome()

# load the start configuration
initial_configuration = np.load('./data/ISS/Initial_Config.npy')

# Create the cube ensemble with an initial cube configuration.
cubes = ProgrammableCubes(initial_configuration)

# Roll-out the command sequence and calculate the final fitness.
steps_needed = cubes.apply_chromosome(chromosome, verbose = False)

# Calculate the custom fitness
custom_fitness_function(cubes, 'ISS')

0.13513513513513514

##### For fitness evaluation on optimize, only the score returned by the fitness function provided in the UDP (and documented on optimize) counts!